In [2]:
from Robinhood import Robinhood
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import time
import datetime as dt

==================================================execution============================================================

In [3]:
#technical indicators
#column names: time, open_price, close_price, high_price, low_price, volume

#Average True Range
def atr(df,n):
    df['h-l']=abs(df['high_price']-df['low_price'])
    df['h-pc']=abs(df['high_price']-df['close_price'].shift(1))
    df['l-pc']=abs(df['low_price']-df['close_price'].shift(1))
    df['tr']=df[['h-l','h-pc','l-pc']].max(axis=1,skipna=False)
    df['atr'] = df['tr'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span=n,adjust=False,min_periods=n).mean()
    #df = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df['atr']

#cumulative annual growth rate
def cagr(df):
    #df['ret'] = df['close_price'].pct_change()
    df["cum_return"] = (1 + df["ret"]).cumprod()
    n = len(df)/len(df)
    cagr = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return cagr

#volatility
def volatility(df):
    #df['ret'] = df['close_price'].pct_change()
    vol = df["ret"].std() * np.sqrt(len(df))
    return vol

#sharpe ratio, rf is the risk free ratio

def sharpe(df, rf):
    sr = (cagr(df) - rf)/volatility(df)
    return sr

def max_dd(df):
    df["cum_return"] = (1 + df["ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

#get historical trade data
def get_ohlcv(robinhood_client, tickers, itvl, prd): #tickers is a list of tickers, itvl: interval, prd: period
    ohlcv = {}
    for ticker in tickers:
        print(ticker)
        data = robinhood_client.get_historical_quotes(ticker, itvl, prd)['results'][0]['historicals']
        df = pd.DataFrame(data = data)
        df.columns.values[0] = "time"
        df["time"] = pd.to_datetime(df['time'])  #'%Y-%m-%dT%H:%M:%SZ
        df = df.set_index("time")
        df.index = df.index.tz_convert('US/Eastern')
        df = df.iloc[:, :-2]
        df[['open_price', 'close_price', 'high_price', 'low_price']] = df[['open_price', 'close_price', 'high_price', 'low_price']].apply(pd.to_numeric)

        df['atr'] = atr(df, 20)
        df['roll_max_cp'] = df['high_price'].rolling(20).max()
        df['roll_min_cp'] = df['low_price'].rolling(20).max()
        df['roll_max_vol'] = df['volume'].rolling(20).max()
        ohlcv[ticker] = df
    return ohlcv

In [4]:
#generate trade signals
def set_signal(tickers):
    tickers_signal = {}
    for ticker in tickers:
        tickers_signal[ticker] = ''
    return tickers_signal

def set_ret(tickers):
    tickers_ret = {}
    for ticker in tickers:
        tickers_ret[ticker] = []
    return tickers_ret

def set_history(tickers):
    tickers_hist = {}
    for ticker in tickers:
        tickers_hist[ticker] = []
    return tickers_hist

def set_latest_buy(tickers):
    tickers_latest_buy = {}
    for ticker in tickers:
        tickers_latest_buy[ticker] = -1
    return tickers_latest_buy

def set_latest_sell(tickers):
    tickers_latest_sell = {}
    for ticker in tickers:
        tickers_latest_sell[ticker] = -1
    return tickers_latest_sell

def resistance_signal(ohlcv, tickers, portfolio, tickers_latest_buy, tickers_latest_sell):

    tickers_signal = set_signal(tickers)
    #tickers_hist = portfolio
    #tickers_latest_buy = set_latest_buy(my_tickers)
    #tickers_latest_sell = set_latest_sell(my_tickers)

    for ticker in tickers:
        if tickers_signal[ticker] == "":
            if ohlcv[ticker]["high_price"].iloc[-1]>=ohlcv[ticker]["roll_max_cp"].iloc[-1] and \
               ohlcv[ticker]["volume"].iloc[-1]>1.5*ohlcv[ticker]["roll_max_vol"].iloc[-2] and \
                tickers_latest_sell[ticker] < ohlcv[ticker].index[-1].day and \
                ohlcv[ticker]["close_price"][-1] * (portfolio[ticker] + 1) <= 100:
                tickers_signal[ticker] = "buy"
                tickers_latest_buy[ticker] = ohlcv[ticker].index[-1].day 
                #tickers_hist[ticker].append(1)
                portfolio[ticker] += 1
            elif ohlcv[ticker]["low_price"].iloc[-1]<=ohlcv[ticker]["roll_min_cp"].iloc[-1] and \
               ohlcv[ticker]["volume"].iloc[-1]>1.5*ohlcv[ticker]["roll_max_vol"].iloc[-2] and \
                portfolio[ticker] > 0 and \
                tickers_latest_buy[ticker] < ohlcv[ticker].index[-1].day:
                tickers_signal[ticker] = "sell"
                tickers_latest_sell[ticker] = ohlcv[ticker].index[-1].day
                #tickers_hist[ticker].append(-1)
            else: 
                pass
                #tickers_hist[ticker].append(0)

        elif tickers_signal[ticker] == "buy":
            if ((ohlcv[ticker]["close_price"][-1]<ohlcv[ticker]["close_price"][-2] - ohlcv[ticker]["atr"][-2]) |
                ohlcv[ticker]["close_price"][-1] * (portfolio[ticker] + 1) > 100):
                tickers_signal[ticker] = ""
                #tickers_hist[ticker].append(0)
            elif ohlcv[ticker]["low_price"].iloc[-1]<=ohlcv[ticker]["roll_min_cp"].iloc[-1] and \
               ohlcv[ticker]["volume"].iloc[-1]>1.5*ohlcv[ticker]["roll_max_vol"].iloc[-2] and \
                portfolio[ticker] > 0 and \
                tickers_latest_buy[ticker] < ohlcv[ticker].index[-1].day:
                tickers_signal[ticker] = "sell"
                tickers_latest_sell[ticker] = ohlcv[ticker].index[-1].day #prevent pattern day trader flag
                #tickers_hist[ticker].append(-1)
            else:
                tickers_latest_buy[ticker] = ohlcv[ticker].index[-1].day #prevent pattern day trader flag
                #tickers_hist[ticker].append(1)
                print('hello')

        elif tickers_signal[ticker] == "sell" and portfolio[ticker] > 0:
            if ohlcv[ticker]["close_price"].iloc[-1] > ohlcv[ticker]["close_price"].iloc[-2] + ohlcv[ticker]["atr"].iloc[-2]:
                tickers_signal[ticker] = ""
                #tickers_hist[ticker].append(0)
            elif ohlcv[ticker]["high_price"].iloc[-1] >= ohlcv[ticker]["roll_max_cp"].iloc[-1] and \
               ohlcv[ticker]["volume"].iloc[-1] > 1.5*ohlcv[ticker]["roll_max_vol"].iloc[-2] and \
                tickers_latest_sell[ticker] < ohlcv[ticker].index[-1].day and \
                ohlcv[ticker]["close_price"][i] * (portfolio[ticker] + 1) <= 100:
                tickers_signal[ticker] = "buy"
                tickers_latest_buy[ticker] = ohlcv[ticker].index[i].day #prevent pattern day trader flag
                #tickers_hist[ticker].append(1)
            else:
                tickers_latest_sell[ticker] = ohlcv[ticker].index[-1].day #prevent pattern day trader flag
                #tickers_hist[ticker].append(-1)

        elif tickers_signal[ticker] == "sell" and portfolio[ticker] <= 0:
            if ohlcv[ticker]["close_price"].iloc[-1] > ohlcv[ticker]["close_price"].iloc[-2] + ohlcv[ticker]["atr"].iloc[-2]:
                tickers_signal[ticker] = ""
                #tickers_hist[ticker].append(0)
            elif ohlcv[ticker]["high_price"].iloc[-1] >= ohlcv[ticker]["roll_max_cp"].iloc[-1] and \
                ohlcv[ticker]["volume"].iloc[-1] > 1.5 * ohlcv[ticker]["roll_max_vol"].iloc[-2] and \
                tickers_latest_sell[ticker] < ohlcv[ticker].index[-1].day and \
                ohlcv[ticker]["close_price"][i] * (portfolio[ticker] + 1) <= 100:
                tickers_signal[ticker] = "buy"
                tickers_latest_buy[ticker] = ohlcv[ticker].index[-1].day #prevent pattern day trader flag
                #tickers_hist[ticker].append(1)
            else: 
                tickers_signal[ticker] = ""
                #tickers_hist[ticker].append(0)

    return tickers_signal

In [5]:
# Current positions and history positions
def get_portfolio(robinhood_client, tickers):
    position_list = robinhood_client.positions()['results']
    portfolio = {}
    for pos in position_list:
        #get ticker
        instr_url = pos['instrument']
        response = requests.get(instr_url)
        json_data = json.loads(response.text)
        ticker = json_data['symbol']

        portfolio[ticker] = float(pos['quantity'])

    for ticker in tickers:
        if ticker in portfolio.keys():
            pass
        else:
            portfolio[ticker] = 0
        
    return portfolio

def get_latest_buy(tickers):
    latest_buy = {}
    yesterday_date = dt.datetime.now().day - 1
    for i in tickers:
        latest_buy[i] = yesterday_date
    return latest_buy

def get_latest_sell(tickers):
    latest_sell = {}
    yesterday_date = dt.datetime.now().day - 1
    for i in tickers:
        latest_sell[i] = yesterday_date
    return latest_sell

In [6]:
def main(my_tickers, robinhood_client,  my_portfolio, my_latest_buy, my_latest_sell):
    ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')
#     my_portfolio = get_portfolio(robinhood_client, my_tickers) #return a dictionary of portfolios with format {ticker: quantity}
#     my_latest_buy = get_latest_buy(my_tickers)
#     my_latest_sell = get_latest_sell(my_tickers)
    print('==============================================data import completed==============================================')
    signals = resistance_signal(ohlcv, my_tickers, my_portfolio, my_latest_buy, my_latest_sell)
    for ticker in my_tickers:
        if signals[ticker] == 'buy':
            stock_instrument = robinhood_client.instruments(ticker)[0]
            #my_portfolio[ticker] += 1
            buy_order = robinhood_client.place_market_buy_order(stock_instrument['url'], ticker, 'GFD', 1)
            #my_latest_buy[ticker] = dt.datetime.now().day
            print('New buy position for ' + ticker)

        elif signals[ticker] == 'sell':
            stock_instrument = robinhood_client.instruments(ticker)[0]
            #my_portfolio[ticker] -= 1
            sell_order = robinhood_client.place_market_sell_order(stock_instrument['url'], ticker, 'GFD', 1)
            #my_latest_sell[ticker] = dt.datetime.now().day
            print('New sell position for ' + ticker)

        else: 
            print('No signals for ' + ticker)
            
    return signals



In [7]:
# my_tickers  = ['GRPN', 'TDC', 'UFS', 'SG', 'TRUE', 'NUZE', 'RC', 'SSTI', 'OCFT', 'YELP', 'VIVO', 'PETQ', 'PDFS', 'NH',
#               'HL', 'F', 'FXI', 'APHA', 'LVS', 'FCX', 'NOK', 'NOG', 'PCG', 'CMCSA', 'KTOS', 'TBIO', 'MOD',
#               'OVID', 'MRSN', 'TGTX', 'GOSS', 'ROAD', 'ESNT', 'CLLDY', 'ERF', 'KT', 'INSW', 'RDN']

#my_tickers = ['BEKE', 'TLSA', 'JMIA', 'GRWG', 'REDU', 'CLVS', 'PROG', 'CPE']

# my_tickers = ['VNET', 'SNPS', 'TJX', 'MLCO', 'QIWI', 'VIPS', 'KEYS', 'BZUN', 'LOW', 
#               'ROST', 'BJ', 'MBT', 'FLNG', 'CRH', 'ADI', 'DE', 'HMLP', 'NDSN', 
#              'CRBP', 'NRZ', 'PTON', 'GM', 'UBER', 'SNE', 'UBER', 'APHA', 'DKNG', 'KO', 'ZNGA', 'PLUG', 'ET', 'DAL']

my_tickers = ['SAIC', 'IMAB', 'VRNT', 'SPWH', 'JAMF', 'CTLT', 'SMAR', 'PD', 'JKS', 'CPRX', 'CRBP', 'VKTX', 'EVRI', 
             'RIGL', 'MTNB', 'GERN', 'GRWG', 'OVID', 'APPS', 'KTOS', 'CBAY', 'AUPH', 'TBIO', 'MNLO', 'MRNS', 'PRPL', 'MEIP', 'AQST', 'HARP']
# Continuous execution        
starttime=time.time()
timeout = time.time() + 60*60*8  # 60 seconds times 60 meaning the script will run for 8 hr
robinhood_client = Robinhood()
robinhood_client.login(username = 'yiwenluo1@outlook.com', password = 'lyw729rolling')
my_portfolio = get_portfolio(robinhood_client, my_tickers) #return a dictionary of portfolios with format {ticker: quantity}
my_latest_buy = get_latest_buy(my_tickers)
my_latest_sell = get_latest_sell(my_tickers)
while time.time() <= timeout: 
    try:
        print("passthrough at ",time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        result = main(my_tickers, robinhood_client, my_portfolio, my_latest_buy, my_latest_sell)
        for ticker in my_tickers:
            if result[ticker] == 'buy':
                my_portfolio[ticker] += 1
                my_latest_buy[ticker] = dt.datetime.now().day
            elif result[ticker] == 'sell':
                my_portfolio[ticker] -= 1
                my_latest_sell[ticker] = dt.datetime.now().day
            else:
                pass
        time.sleep(150 - ((time.time() - starttime) % 150.0)) # 5 minute interval between each new execution
    except KeyboardInterrupt:
        print('\n\nKeyboard exception received. Exiting.')
        exit()

In [ ]:
robinhood_client = Robinhood()
robinhood_client.login(username = 'yiwenluo1@outlook.com', password = 'lyw729rolling')

In [ ]:

ohlcv = get_ohlcv(robinhood_client, my_tickers, '5minute', 'week')

In [ ]:
for i in my_tickers:
    ohlcv[i] = ohlcv[i][:-4]


In [ ]:
ohlcv['UAA']

In [ ]:
#my_tickers = ['OXY', 'BKR', 'NWL', 'MO', 'DISCA', 'NLOK', 'MRO', 'TWTR', 'BEN', 'IPG']
my_tickers  = ['NOK', 'SIRI', 'NOG', 'PCG', 'RWT', 'FCAU', 'CMCSA', 'ICAGY', 'TBIO', 'KTOS']
my_portfolio = get_portfolio(robinhood_client, my_tickers) #return a dictionary of portfolios with format {ticker: quantity}
my_latest_buy = get_latest_buy(my_tickers)
my_latest_sell = get_latest_sell(my_tickers)
signals = resistance_signal(ohlcv, my_tickers, my_portfolio, my_latest_buy, my_latest_sell)

In [ ]:
my_latest_buy

In [ ]:
pd.set_option('display.max_rows', 500)
ohlcv['UAA']

In [ ]:
tickers_hist['DXC']

In [ ]:
signals

In [ ]:
robinhood_client.portfolios()

In [ ]:
get_pflio(robinhood_client)

In [ ]:
json_data = json.loads(response.text)

In [ ]:
json_data

In [ ]:
to_datetime(robinhood_client.positions()['results'][0]['updated_at'])

In [ ]:
dt.datetime.now().day-1

In [ ]:
No 2FA Given
SMS Code:
325866
passthrough at  2020-07-24 13:47:35
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 13:52:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 13:57:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:02:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:07:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:12:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:17:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:22:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:27:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:32:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:37:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:42:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:47:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:52:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
No signals for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 14:57:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
No signals for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.16, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:02:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.64, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.65, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:07:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.65, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:12:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.51, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:17:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.41, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:22:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 41.99, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:27:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 41.99, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:32:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 41.99, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:37:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:42:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 41.99, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:47:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:52:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 15:57:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:02:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.35, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:07:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.25, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:12:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.25, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:17:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.26, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:22:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.26, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:27:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.26, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:32:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.26, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:37:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.65, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:42:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.65, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG
passthrough at  2020-07-24 16:47:22
==============================================data import completed==============================================
No signals for OXY
No signals for BKR
No signals for NWL
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/84719532-6913-4a1e-a943-f2947f822715/', 'symbol': 'MO', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 42.0, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for MO
{'account': 'https://api.robinhood.com/accounts/493282446/', 'instrument': 'https://api.robinhood.com/instruments/ed19f28c-9172-4d84-8752-719e0239ad13/', 'symbol': 'DISCA', 'type': 'market', 'time_in_force': 'gfd', 'trigger': 'immediate', 'price': 21.65, 'quantity': 1, 'side': 'buy'}
400 Client Error: Bad Request for url: https://api.robinhood.com/orders/
New buy position for DISCA
No signals for NLOK
No signals for MRO
No signals for TWTR
No signals for BEN
No signals for IPG

In [ ]:
def test():
    print('test')
    test = 'a'
    return test
result = test()

In [ ]:
result